In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from keras.preprocessing import image
from keras.models import model_from_json
import pickle
import torchvision
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input

In [2]:
num_classes = 10
input_shape = (28, 28, 1)

In [3]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [4]:
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [5]:
np.shape(x_test)

(10000, 28, 28, 1)

In [8]:
#model = torchvision.models.vgg19(pretrained=True)

In [13]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

In [16]:
batch_size = 500
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
108/108 [==============================] - 19s 171ms/step - loss: 0.2240 - accuracy: 0.9317 - val_loss: 0.0955 - val_accuracy: 0.9738
Epoch 2/15
108/108 [==============================] - 14s 133ms/step - loss: 0.1323 - accuracy: 0.9592 - val_loss: 0.0740 - val_accuracy: 0.9785
Epoch 3/15
108/108 [==============================] - 15s 138ms/step - loss: 0.1049 - accuracy: 0.9678 - val_loss: 0.0578 - val_accuracy: 0.9835
Epoch 4/15
108/108 [==============================] - 15s 141ms/step - loss: 0.0861 - accuracy: 0.9732 - val_loss: 0.0514 - val_accuracy: 0.9857
Epoch 5/15
108/108 [==============================] - 14s 133ms/step - loss: 0.0760 - accuracy: 0.9763 - val_loss: 0.0457 - val_accuracy: 0.9877
Epoch 6/15
108/108 [==============================] - 14s 130ms/step - loss: 0.0680 - accuracy: 0.9787 - val_loss: 0.0428 - val_accuracy: 0.9878
Epoch 7/15
108/108 [==============================] - 14s 129ms/step - loss: 0.0642 - accuracy: 0.9798 - val_loss: 0.0409 - val_ac

In [17]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.028172805905342102
Test accuracy: 0.9901000261306763


In [18]:
#Prediction part
test_image = image.load_img("D:/Clemson/COURSE/MNIST API/TestData/testSet/testSet/img_13.jpg", target_size = (28, 28,1),grayscale=True)
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
test_image = test_image.astype("float32") / 255
np.shape(test_image[0])

E:\anaconda3\envs\CPSC-8430-DeepLearning\lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


(28, 28, 1)

In [25]:
result = model.predict(test_image)
# print(np.max(result))
# print(result[0])
for i in range(10):
    if result[0][i] == np.max(result):
        prediction = i
print(f"Prediction of given input : {prediction}")

Prediction of given input : 4


In [27]:
#Pickle Dump
pickle_out = open("MNIST_PICKEL.pkl","wb")
pickle.dump (model,pickle_out)
pickle_out.close()

INFO:tensorflow:Assets written to: ram://bce26767-fa11-4d61-a3b6-d164258d33b9/assets


NotFoundError: 

In [ ]:
#saving model 
model_json = model.to_json()
with open("mnist.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("mnist.h5")
print("Saved mnist model to disk")

Saved mnist model to disk


In [ ]:
# load json and create model
json_file = open('mnist.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("mnist.h5")
print("Loaded mnist model from disk")

Loaded mnist model from disk


In [ ]:
result = loaded_model.predict(test_image)
for i in range(10):
    if result[0][i] == np.max(result):
        print(f'Predicted Values is : {i}')

Predicted Values is : 4
